In [245]:
import tensorflow as tf
import numpy as np
import pandas as pd
from olddatasetclass import Dataset
from oldmodel import get_model, Args
import math

In [75]:
args = Args()
num_users = 943
num_items = 1682
model = get_model(num_users, num_items, args.num_factors, 
                  eval(args.layers), eval(args.reg_layers), eval(args.reg_mf))

In [76]:
dataset = Dataset(args.path, args.dataset)
train, testRatings, testNegatives = dataset.train_ratings, dataset.test_ratings, dataset.negatives

In [119]:
testNegatives_array = testNegatives.iloc[:, 1].tolist()
testNegatives_array = np.asarray(testNegatives_array)
print(testRatings_array[0:5])
testNegatives_array.shape

[[ 590  299 1252 1060  841 1127  779  952  839 1246 1097  631 1323  930
   853  316  420 1635  719  543 1131  506  672 1325 1024  660  546  462
  1443 1174  637  817  729 1462 1601 1650  576 1204 1544  770  594  572
   569 1554  556  407 1082  317  422  761 1617  907  275  434 1591  804
   610 1069  983 1600 1436 1456  302 1569 1360  636  675 1002  791  708
  1136 1665  442 1597 1616  790  908  892 1680 1013 1012  289  604  310
   685 1589  912 1489  430  573  806  901  970  944  668 1469  315 1608
  1334]
 [  44  672 1577  390 1148  450  167 1262  631  748  549 1188  445  472
  1518  669  976   74  661  766 1637  237  478  466 1172  321 1567 1650
  1627 1314 1488  726  139  587  523  147  473 1383 1365 1132   61 1446
  1080 1503  673  603 1018  607 1176  875 1411  776 1034 1286 1194 1302
   997  940 1473   93  327  659 1119 1531 1371  711   69 1033 1131 1284
   173  567  425 1447  761  553  903 1281 1584  815  395  918  584 1660
   744 1029 1316  207 1494 1327 1057  231 1078  540 1375

(943, 99)

In [120]:
testRatings_array = testRatings.iloc[:, 1].values.reshape((-1, 1))
print(testRatings_array[0:5])
testRatings_array.shape

[[101]
 [280]
 [319]
 [ 10]
 [456]]


(943, 1)

In [172]:
testSamples = np.concatenate((testRatings_array, testNegatives_array), axis=1)
print(testSamples)
testSamples.shape

[[ 101  590  299 ...  315 1608 1334]
 [ 280   44  672 ...  634  536 1576]
 [ 319  673 1547 ...  526 1085  623]
 ...
 [   0  903  345 ... 1629  418  663]
 [ 661 1175  956 ...  490  885  124]
 [ 233 1268 1243 ... 1101  695  178]]


(943, 100)

In [173]:
shape = testSamples.shape
testSamples = testSamples.reshape(-1, 1)
userSamples = np.asarray(range(shape[0])).repeat(shape[1])

Initiate inputs with `testSamples`

In [182]:
predictions = model.predict([userSamples, testSamples], batch_size=256)
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# sess.run(out)
predictions = predictions.reshape(shape)
print(predictions)
predictions.shape

[[0.5036962  0.5019878  0.50089216 ... 0.5016976  0.5032916  0.50249535]
 [0.50528663 0.5036458  0.50047773 ... 0.5022622  0.50067204 0.5034733 ]
 [0.50134766 0.49868655 0.49893048 ... 0.49891677 0.50242317 0.49997872]
 ...
 [0.5006457  0.5015409  0.5014249  ... 0.4999535  0.5006036  0.50078374]
 [0.5052906  0.50257695 0.5045334  ... 0.50702035 0.5022472  0.50407714]
 [0.5073326  0.50071734 0.50729036 ... 0.5033837  0.5038245  0.506637  ]]


(943, 100)

In [210]:
def isFirstK(array, K=10):
    """check if the first element ranks in first K
    """
    # argsort find the index of K largest elements
    topK = array.argsort()[-K:]
    # find if 0 is in the largest K
    out = 1 if (0 in topK) else 0
    return out

In [211]:
out = np.apply_along_axis(isFirstK, axis=1, arr=predictions)
out.shape

(943,)

In [214]:
sum(out)/len(out)

0.0975609756097561

In [215]:
topK = np.flip(array.argsort()[-K:])

In [216]:
topK

array([38, 92, 55, 43, 66, 54, 76, 20, 37, 63])

In [217]:
array.argsort()[-K:]

array([63, 37, 20, 76, 54, 66, 43, 55, 92, 38])

In [247]:
a,= np.where(array.argsort()[-K:] == 63)

In [248]:
math.log(2) / math.log(a[0] + 2)

1.0

In [237]:
a

array([0])

In [238]:
a[0] is None

False

In [242]:
len(a)

0